# Up and going with tf( version 1)

## Imports

In [9]:
import tensorflow.compat.v1 as tf
import numpy
from sklearn.datasets import fetch_california_housing

2024-02-07 12:58:45.384006: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 12:58:45.424858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
tf.disable_eager_execution() 

In [3]:
x = tf.Variable(3,name='x')
y = tf.Variable(8,name='y')

In [4]:
f = x*x*y + y + 5

In [5]:
sess = tf.Session()

In [6]:
sess.run(x.initializer)

2024-02-06 21:45:09.051701: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [7]:
sess.run(y.initializer)

In [8]:
result = sess.run(f)

In [9]:
result

85

In [10]:
sess.close()

In [11]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    res = sess.run(f)

In [12]:
res

85

In [13]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    res = f.eval()

In [14]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    res = f.eval()

In [15]:
res

85

In [16]:
x1 = tf.Variable(1)

In [17]:
x1.graph

In [18]:
tf.get_default_graph()

In [19]:
x1.graph is tf.get_default_graph()

True

In [20]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(3)
    

In [21]:
x2.graph is graph

True

In [22]:
x2.graph is tf.get_default_graph()

False

In [23]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval()) 

10
15


In [24]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val) # 10
    print(z_val) # 15

10
15


In [12]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = numpy.c_[numpy.ones((m, 1)), housing.data]


In [13]:
m,n

(20640, 8)

In [27]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [14]:
scaled_housing_data_plus_bias = housing_data_plus_bias.copy()
means,stds = housing_data_plus_bias[:,1:].mean(axis=0),housing_data_plus_bias[:,1:].std(axis=0)
scaled_housing_data_plus_bias[:,1:] -= means
scaled_housing_data_plus_bias[:,1:] /= stds

In [29]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            sess.run(training_op)
            best_theta = theta.eval()

Epoch 0 MSE = 7.565616
Epoch 100 MSE = 7.2847657
Epoch 200 MSE = 7.016872
Epoch 300 MSE = 6.7612643
Epoch 400 MSE = 6.517311
Epoch 500 MSE = 6.2844157
Epoch 600 MSE = 6.0620193
Epoch 700 MSE = 5.8495903
Epoch 800 MSE = 5.64663
Epoch 900 MSE = 5.452666


In [30]:
A = tf.placeholder(tf.float32,shape=(None,3))
B = A +5

with tf.Session() as sess:
    B1 = B.eval(feed_dict={A:[[1,2,3],[7,8,9]]})

In [31]:
B1

array([[ 6.,  7.,  8.],
       [12., 13., 14.]], dtype=float32)

# Mini Batch Gradient Descent

## Unscaled data( Exploding Gradient)

In [32]:
def fetch_mini_batch(ix,size):
    ix   = slice(size*ix,size*(ix+1))
    data = lambda : fetch_california_housing()
    y = data().target[ix].reshape(-1,1)
    X = data().data[ix,:]
    bias = numpy.ones((min(size,len(X)), 1))
    X = numpy.c_[bias, X]
    return y,X     

In [33]:
(r:=fetch_mini_batch(1,5),r[0].shape,r[1].shape)

((array([[2.697],
         [2.992],
         [2.414],
         [2.267],
         [2.611]]),
  array([[ 1.00000000e+00,  4.03680000e+00,  5.20000000e+01,
           4.76165803e+00,  1.10362694e+00,  4.13000000e+02,
           2.13989637e+00,  3.78500000e+01, -1.22250000e+02],
         [ 1.00000000e+00,  3.65910000e+00,  5.20000000e+01,
           4.93190661e+00,  9.51361868e-01,  1.09400000e+03,
           2.12840467e+00,  3.78400000e+01, -1.22250000e+02],
         [ 1.00000000e+00,  3.12000000e+00,  5.20000000e+01,
           4.79752705e+00,  1.06182380e+00,  1.15700000e+03,
           1.78825348e+00,  3.78400000e+01, -1.22250000e+02],
         [ 1.00000000e+00,  2.08040000e+00,  4.20000000e+01,
           4.29411765e+00,  1.11764706e+00,  1.20600000e+03,
           2.02689076e+00,  3.78400000e+01, -1.22260000e+02],
         [ 1.00000000e+00,  3.69120000e+00,  5.20000000e+01,
           4.97058824e+00,  9.90196078e-01,  1.55100000e+03,
           2.17226891e+00,  3.78400000e+01, -1.222

In [18]:
batch_size = 1000
n_batches  = int(numpy.ceil(m/batch_size))

In [35]:
y = tf.placeholder(tf.float32,shape=(None,1),name='y')
X = tf.placeholder(tf.float32,shape=(None,n+1),name='X')

In [15]:
lr = .01

In [37]:
θ = tf.Variable(numpy.zeros((n+1,1),dtype='float32'),name='Theta')
yhat = tf.matmul(X,θ,name='predictions')

ε = yhat - y

mse = tf.reduce_mean(tf.square(ε),name='mse')

gradients = (2/m)*tf.matmul(tf.transpose(X),ε)

trainop = tf.assign(θ,θ-lr*gradients)

In [38]:
n_epochs = 200
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_ix in range(n_batches):
            y_batch,X_batch = fetch_mini_batch(batch_ix,batch_size)
            sess.run(trainop,feed_dict={y:y_batch,X:X_batch})
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={y:y_batch,X:X_batch}))
    best_theta = θ.eval()

Epoch 0 MSE = nan
Epoch 100 MSE = nan


## Scaled Data

In [16]:
def fetch_mini_batch(ix,size):
    ix   = slice(size*ix,size*(ix+1))
    data = lambda : fetch_california_housing()
    y = data().target[ix].reshape(-1,1)
    X = (data().data[ix,:]-means)/stds
    bias = numpy.ones((min(size,len(X)), 1))
    X = numpy.c_[bias, X]
    return y,X    

In [ ]:
θ = tf.Variable(numpy.zeros((n+1,1),dtype='float32'),name='Theta')
yhat = tf.matmul(X,θ,name='predictions')

ε = yhat - y

mse = tf.reduce_mean(tf.square(ε),name='mse')

gradients = (2/m)*tf.matmul(tf.transpose(X),ε)

trainop = tf.assign(θ,θ-lr*gradients)

n_epochs = 1000
init = tf.global_variables_initializer()

saver = tf.train.Saver() #saving sessions

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        
        for batch_ix in range(n_batches):
            
            y_batch,X_batch = fetch_mini_batch(batch_ix,batch_size)
            
            sess.run(trainop,feed_dict={y:y_batch,X:X_batch})
            
        if epoch % 100 == 0:
                saver.save(sess,f"models/model.ckpt")
                print("Epoch", epoch, "MSE =", mse.eval(feed_dict={y:y_batch,X:X_batch}))
    bestθ = θ.eval()
    saver.save(sess,f"models/modelfinal.ckpt")


In [ ]:
bestθ

# Visializing with tensorboard

In [7]:
import datetime
now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
logd = f'tf_logs/run-{now}/'
logd

'tf_logs/run-2024-02-07-12-55-45/'

In [22]:
graph = tf.Graph()

with graph.as_default():
    
    y = tf.placeholder(tf.float32,shape=(None,1),name='y')
    X = tf.placeholder(tf.float32,shape=(None,n+1),name='X')
    
    θ = tf.Variable(numpy.zeros((n+1,1),dtype='float32'),name='Theta')
    yhat = tf.matmul(X,θ,name='predictions')
    
    ε = yhat - y
    
    mse = tf.reduce_mean(tf.square(ε),name='mse')
    
    gradients = (2/m)*tf.matmul(tf.transpose(X),ε) 
    
    trainop = tf.assign(θ,θ-lr*gradients)
    
    n_epochs = 1000
    init = tf.global_variables_initializer()
        
    mse_summary = tf.summary.scalar('mse',mse)
    

    file_writer = tf.summary.FileWriter(logd,graph=graph)
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(n_epochs):
            
            for batch_ix in range(n_batches):
                
                y_batch,X_batch = fetch_mini_batch(batch_ix,batch_size)
                
                sess.run(trainop,feed_dict={y:y_batch,X:X_batch})
                
                summary = mse_summary.eval(feed_dict={y:y_batch,X:X_batch})
                
                file_writer.add_summary(summary,epoch*batch_ix)

        
        bestθ = θ.eval()

In [20]:
help(tf.summary)

Help on package tensorflow._api.v2.compat.v1.summary in tensorflow._api.v2.compat.v1:

NAME
    tensorflow._api.v2.compat.v1.summary - Public API for tf._api.v2.summary namespace

PACKAGE CONTENTS


CLASSES
    builtins.object
        tensorflow.python.summary.writer.writer_cache.FileWriterCache
    google._upb._message.Message(builtins.object)
        tensorflow.core.framework.summary_pb2.Summary(google._upb._message.Message, google.protobuf.message.Message)
        tensorflow.core.framework.summary_pb2.SummaryDescription(google._upb._message.Message, google.protobuf.message.Message)
        tensorflow.core.util.event_pb2.Event(google._upb._message.Message, google.protobuf.message.Message)
        tensorflow.core.util.event_pb2.SessionLog(google._upb._message.Message, google.protobuf.message.Message)
        tensorflow.core.util.event_pb2.TaggedRunMetadata(google._upb._message.Message, google.protobuf.message.Message)
    google.protobuf.message.Message(builtins.object)
        tensor

In [44]:
%ls tf_logs/

run-2024-02-06-21-07-40/  run-2024-02-06-21-19-57/  run-2024-02-06-21-48-05/


In [45]:
%ls {logd}

events.out.tfevents.1707256086.588a547f6c7a
events.out.tfevents.1707256124.588a547f6c7a


In [46]:
file_writer

In [47]:
file_writer.close()

In [48]:
%ls -la {logd}

total 932
drwxr-xr-x 1 analyst analyst    512 Feb  6 21:48 ./
drwxr-xr-x 1 analyst analyst    512 Feb  6 21:48 ../
-rw-r--r-- 1 analyst analyst  18039 Feb  6 21:48 events.out.tfevents.1707256086.588a547f6c7a
-rw-r--r-- 1 analyst analyst 921012 Feb  6 21:54 events.out.tfevents.1707256124.588a547f6c7a


In [24]:
%%HTML
<iframe src=http://localhost:6006/ style="width: 1000px; height: 600px;"></>